<h1>Snow-Related 311 Complaints in Chicago, 2019-2023</h1>
11 January 2024

This analysis looks at all four types of snow-related 311 complaints:
<table>
    <tr><td><strong>SR_SHORT_CODE</strong></td><td><strong>Complaint Description</strong></td><td><strong>Responsible Department</strong></td></tr>
    <tr><td>SWSNOREM</td><td>Snow – Uncleared Sidewalk Complaint</td><td>Streets and Sanitation</td></tr>
    <tr><td>SDO</td><td>Ice and Snow Removal Request</td><td>Streets and Sanitation</td></tr>
    <tr><td>SDW</td><td>Object/Dibs Removal Request</td><td>CDOT</td></tr>
    <tr><td>SNPBLBS</td><td>Snow Removal - Protected Bike Lane or Bridge Sidewalk</td><td>CDOT</td></tr> 
</table>

My analysis steps:
<ol>
<li><a href="#read">Read Data</a>
<li><a href="#tabulate">Tabulate by Community Area</a>
<li><a href="#summarize">Summarize Results</a>
</ol>

<br>
Reminder that snow "seasons" run from July 1 to June 30 of the following year. For example, in my processed data set the season '2020-2021' refers to July 1, 2020 to June 30, 2021. 

<h3>Preliminary Findings</h3>
<ul>
    <li>The five neighborhoods with the most 311 complaints per capita are (in order) Lincoln Square, Logan Square, Uptown, West Town, Lincoln Park
        <li>The five neighborhoods with the least 311 complaints per capita are (in order) West Pullman, Riverdale, Mount Greenwood, Pullman, Hegewisch. These all seem to be car-centric neighborhoods.
</ul>

<a name="read"></a>
# 1. Read Data

In [1]:
import pandas as pd
import requests
#import datetime as dt #would only need this if I manipulated dates post-API data retrieval

### Read snow violations

In [2]:
df = pd.read_csv("../data/311-snow-violations.csv")
df.head()

,SR_NUMBER,SR_SHORT_CODE,CREATED_DATE,STREET_ADDRESS,COMMUNITY_AREA,WARD,STATUS,ORIGIN,CLOSED_DATE,LATITUDE,LONGITUDE,SR_TYPE,year,month,date,season,GEOID,COMMUNITY_NAME
0,SR21-00149474,SDO,2021-01-27 13:13:50,1700 W 15TH ST,28,28,Completed,Mobile Device,2021-01-27 19:50:47,41.861457,-87.668881,Ice and Snow Removal Request,2021,1,2021-01-27,2020-2021,28,Near West Side
1,SR21-00177207,SDO,2021-02-01 10:19:34,1300 S HEATH AVE,28,28,Completed,Mobile Device,2021-02-01 15:12:41,41.864744,-87.684402,Ice and Snow Removal Request,2021,2,2021-02-01,2020-2021,28,Near West Side
2,SR21-00269268,SDO,2021-02-17 12:43:47,819 S BISHOP ST,28,28,Completed,Mobile Device,2021-02-19 00:58:56,41.871081,-87.662624,Ice and Snow Removal Request,2021,2,2021-02-17,2020-2021,28,Near West Side
3,SR21-00149811,SDO,2021-01-27 13:50:04,1300 S WOLCOTT AVE,28,28,Completed,Mobile Device,2021-01-27 19:10:36,41.864893,-87.673790,Ice and Snow Removal Request,2021,1,2021-01-27,2020-2021,28,Near West Side
4,SR21-00188251,SDO,2021-02-02 15:47:03,1200 W LEXINGTON ST,28,28,Completed,Mobile Device,2021-02-02 21:53:42,41.872287,-87.656946,Ice and Snow Removal Request,2021,2,2021-02-02,2020-2021,28,Near West Side


In [3]:
len(df)

68273

### look at just full years

In [4]:
df = df[(df['season'].isin(['2019-2020','2020-2021','2021-2022','2022-2023']))]
len(df)

67361

### read community population

In [5]:
# retrieved on 1/11/24, but 2020 Census Population figures should be static

df_population = pd.read_csv("../data/population_cmap_2022.csv")

In [6]:
# simplify dataframe to get only essentials
df_population = df_population[['GEOID','GEOG','2020_POP']]
df_population = df_population.rename(columns={'GEOG':'COMMUNITY_NAME'})
df_population['COMMUNITY_CAPS']=df_population['COMMUNITY_NAME'].str.upper()
df_population.head()

,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,14,Albany Park,48396,ALBANY PARK
1,57,Archer Heights,14196,ARCHER HEIGHTS
2,34,Armour Square,13890,ARMOUR SQUARE
3,70,Ashburn,41098,ASHBURN
4,71,Auburn Gresham,44878,AUBURN GRESHAM


<a name="tabulate"></a>
# 2. Tabulate By Community Area by 311 Type, 2019-2023

In [7]:
df_community_by_type = df.pivot_table(index='COMMUNITY_NAME', columns='SR_TYPE', values='SR_NUMBER', aggfunc='size', fill_value=0)
df_community_by_type

SR_TYPE,Ice and Snow Removal Request,Snow - Object/Dibs Removal Request,Snow Removal - Protected Bike Lane or Bridge Sidewalk,Snow – Uncleared Sidewalk Complaint
COMMUNITY_NAME,,,,
Albany Park,540,195,16,400
Archer Heights,205,253,0,60
Armour Square,65,52,16,77
Ashburn,1506,75,1,108
Auburn Gresham,1449,179,1,128
...,...,...,...,...
West Lawn,248,434,0,146
West Pullman,752,16,0,23
West Ridge,505,200,5,804


### merge in community data

In [8]:
df_community_summary = pd.merge(left=df_community_by_type,right=df_population,on='COMMUNITY_NAME')
df_community_summary.head()

,COMMUNITY_NAME,Ice and Snow Removal Request,Snow - Object/Dibs Removal Request,Snow Removal - Protected Bike Lane or Bridge Sidewalk,Snow – Uncleared Sidewalk Complaint,GEOID,2020_POP,COMMUNITY_CAPS
0,Albany Park,540,195,16,400,14,48396,ALBANY PARK
1,Archer Heights,205,253,0,60,57,14196,ARCHER HEIGHTS
2,Armour Square,65,52,16,77,34,13890,ARMOUR SQUARE
3,Ashburn,1506,75,1,108,70,41098,ASHBURN
4,Auburn Gresham,1449,179,1,128,71,44878,AUBURN GRESHAM


In [9]:
# per 10,000 capita, per year over 4 years
df_community_summary['Streets Per 10k'] = \
(10000/4)*df_community_summary['Ice and Snow Removal Request']/df_community_summary['2020_POP']

df_community_summary['Dibs Per 10k'] = \
(10000/4)*df_community_summary['Snow - Object/Dibs Removal Request']/df_community_summary['2020_POP']

df_community_summary['Sidewalks Per 10k'] = \
(10000/4)*df_community_summary['Snow – Uncleared Sidewalk Complaint']/df_community_summary['2020_POP']

df_community_summary['Bike-Bridge Per 10k'] = \
(10000/4)*df_community_summary['Snow Removal - Protected Bike Lane or Bridge Sidewalk']/df_community_summary['2020_POP']

In [23]:
# look at all stats
df_community_summary.head()

,COMMUNITY_NAME,Ice and Snow Removal Request,Snow - Object/Dibs Removal Request,Snow Removal - Protected Bike Lane or Bridge Sidewalk,Snow – Uncleared Sidewalk Complaint,GEOID,2020_POP,COMMUNITY_CAPS,Streets Per 10k,Dibs Per 10k,Sidewalks Per 10k,Bike-Bridge Per 10k
0,Albany Park,540,195,16,400,14,48396,ALBANY PARK,27.894867,10.073147,20.662865,0.826515
1,Archer Heights,205,253,0,60,57,14196,ARCHER HEIGHTS,36.101719,44.554804,10.566357,0.000000
2,Armour Square,65,52,16,77,34,13890,ARMOUR SQUARE,11.699064,9.359251,13.858891,2.879770
3,Ashburn,1506,75,1,108,70,41098,ASHBURN,91.610297,4.562266,6.569663,0.060830
4,Auburn Gresham,1449,179,1,128,71,44878,AUBURN GRESHAM,80.718838,9.971478,7.130443,0.055707


In [11]:
df_community_summary.to_csv("../results/311_community_by_type.csv", index=False)

<a name="#summarize"></a>
# 3. Summarize Stats for Uncleared Sidewalk Complaints

In [17]:
# filter to include only uncleared sidewalks
df_uncleared = df[(df['SR_SHORT_CODE']=='SWSNOREM')]
len(df_uncleared)

21079

### uncleared by season

In [18]:
df_by_season = df_uncleared.groupby('season').agg(complaints=('SR_NUMBER','count')).reset_index()
df_by_season

,season,complaints
0,2019-2020,6541
1,2020-2021,6494
2,2021-2022,6416
3,2022-2023,1628


### uncleared by complaint origin

In [20]:
df_by_type = df_uncleared.groupby('ORIGIN').agg(complaints=('SR_NUMBER','count')).reset_index()
df_by_type

,ORIGIN,complaints
0,Alderman's Office,724
1,E-Mail,22
2,Generated In House,1
3,Internet,7068
4,Mobile Device,8260
5,Open311 Interface,1
6,Phone Call,4818
7,Salesforce Mobile App,36
8,spot-open311-Chicago+Works,97
9,spot-open311-SeeClickFix,52


In [21]:
# consolidate other
df_uncleared['ORIGIN_binned'] = df_uncleared['ORIGIN'].replace(['E-Mail', 'Generated In House', 'Open311 Interface','Salesforce Mobile App','spot-open311-Chicago+Works','spot-open311-SeeClickFix'], 'Other')

/tmp/ipykernel_961/401749550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uncleared['ORIGIN_binned'] = df_uncleared['ORIGIN'].replace(['E-Mail', 'Generated In House', 'Open311 Interface','Salesforce Mobile App','spot-open311-Chicago+Works','spot-open311-SeeClickFix'], 'Other')


In [22]:
df_by_type = df_uncleared.groupby('ORIGIN_binned').agg(complaints=('SR_NUMBER','count')).reset_index()
df_by_type

,ORIGIN_binned,complaints
0,Alderman's Office,724
1,Internet,7068
2,Mobile Device,8260
3,Other,209
4,Phone Call,4818


### uncleared by status

In [18]:
df_by_status = df_uncleared.groupby('STATUS')['SR_NUMBER'].agg('count').reset_index()
df_by_status

,STATUS,SR_NUMBER
0,Canceled,978
1,Completed,20041
2,Open,60


### uncleared rate by community

In [26]:
df_community_summary.head()

,COMMUNITY_NAME,Ice and Snow Removal Request,Snow - Object/Dibs Removal Request,Snow Removal - Protected Bike Lane or Bridge Sidewalk,Snow – Uncleared Sidewalk Complaint,GEOID,2020_POP,COMMUNITY_CAPS,Streets Per 10k,Dibs Per 10k,Sidewalks Per 10k,Bike-Bridge Per 10k
0,Albany Park,540,195,16,400,14,48396,ALBANY PARK,27.894867,10.073147,20.662865,0.826515
1,Archer Heights,205,253,0,60,57,14196,ARCHER HEIGHTS,36.101719,44.554804,10.566357,0.000000
2,Armour Square,65,52,16,77,34,13890,ARMOUR SQUARE,11.699064,9.359251,13.858891,2.879770
3,Ashburn,1506,75,1,108,70,41098,ASHBURN,91.610297,4.562266,6.569663,0.060830
4,Auburn Gresham,1449,179,1,128,71,44878,AUBURN GRESHAM,80.718838,9.971478,7.130443,0.055707


In [35]:
df_uncleared_by_community = df_community_summary[['COMMUNITY_NAME','Sidewalks Per 10k','Ice and Snow Removal Request','2020_POP','COMMUNITY_CAPS']].sort_values(by='Sidewalks Per 10k', ascending=False)
df_uncleared_by_community

,COMMUNITY_NAME,Sidewalks Per 10k,Ice and Snow Removal Request,2020_POP,COMMUNITY_CAPS
39,Lincoln Square,57.354176,440,40494,LINCOLN SQUARE
40,Logan Square,55.361753,762,71665,LOGAN SQUARE
66,Uptown,46.124655,343,57182,UPTOWN
75,West Town,45.824267,1029,87781,WEST TOWN
38,Lincoln Park,42.628951,515,70492,LINCOLN PARK
...,...,...,...,...,...
30,Hegewisch,3.739902,150,10027,HEGEWISCH
57,Pullman,2.565982,299,6820,PULLMAN
45,Mount Greenwood,2.415718,193,18628,MOUNT GREENWOOD
58,Riverdale,2.409804,51,7262,RIVERDALE


In [36]:
df_uncleared_by_community.to_csv("../results/311_uncleared_by_community.csv", index=False)